# NVIDIA TensorRT Engines with TREx

In [4]:
import os
print(os.getcwd())

/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry


## 1. Convert the trained `best_qat.onnx` to `.engine` using `trtexec`

- Example command: 

   ```bash
    trtexec --verbose --onnx=../tests/inputs/mobilenet.qat.onnx --saveEngine=../tests/inputs/mobilenet.qat.onnx.engine --exportLayerInfo=../tests/inputs/mobilenet.qat.onnx.engine.graph.json --timingCacheFile=./timing.cache --profilingVerbosity=detailed --best
   ```

In [5]:
# Add the directory containing trtexec to the PATH
os.environ['PATH'] += os.pathsep + '/usr/src/tensorrt/bin'

# Verify that trtexec is in the PATH
!echo $PATH

# Test if trtexec is accessible
!trtexec --help

/home/jeffrymahbuubi/Syringe-Detection/env/tensorrt/bin:/home/jeffrymahbuubi/.vscode-server/bin/5437499feb04f7a586f677b155b039bc2b3669eb/bin/remote-cli:/usr/local/cuda-11.8/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files (x86)/Windows Kits/10/Windows Performance Toolkit/:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Program Files/NVIDIA Corporation/NVIDIA NvDLISR:/Docker/host/bin:/mnt/c/Program Files/usbipd-win/:/mnt/c/Users/11208120/AppData/Local/Microsoft/WindowsApps:/mnt/c/Program Files/JetBrains/PyCharm 2024.1.3/bin:/mnt/c/ProgramData/miniconda3:/mnt/c/ProgramData/miniconda3/Scripts:/mnt/c/ProgramData/miniconda3/Library/bin:/mnt/c/Users/11208120/AppData/Local/Programs/Microsoft VS Code/bin:/snap/bin:/home/

In [6]:
# Run the trtexec command to generate the .engine and .json files
# !trtexec --verbose --onnx=./run/detect/QAT_b8_e200_q40_relu_rect/weights/best_qat.onnx --saveEngine=./run/detect/QAT_b8_e200_q40_relu_rect/weights/best_qat.engine --int8 --best --device=0

# Using `process_engine.py
!python /home/jeffrymahbuubi/Syringe-Detection/compression/TensorRT/tools/experimental/trt-engine-explorer/utils/process_engine.py /home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/best_qat.onnx /home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/tensorrt best

Building the engine: /home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/tensorrt/best_qat.onnx.engine
trtexec --verbose --onnx=/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/best_qat.onnx --saveEngine=/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/tensorrt/best_qat.onnx.engine --exportLayerInfo=/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/tensorrt/best_qat.onnx.engine.graph.json --timingCacheFile=./timing.cache --profilingVerbosity=detailed --best

Successfully built the engine.

Engine building metadata: generated output file /home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b8_e200_q40_relu_rect/weights/tensorrt/best_qat.onnx.engine.build.metadata.json
Profiling th

## 2. Load `JSON` Files

In [79]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import trex
import trex.notebook
import trex.plotting
import trex.graphing
import trex.df_preprocessing

# Configure a wider output (for the wide graphs)
trex.notebook.set_wide_display()

# Choose an engine file to load. This notebook assumes that you've saved the engine to the following paths.
engine_name = "./run/detect/QAT_b16_e100_silu/weights/tensorrt/best_qat.onnx.engine"

## 3. Create an `EnginePlan` instance and start exploring

In [80]:
assert engine_name is not None
plan = trex.EnginePlan(f'{engine_name}.graph.json', f'{engine_name}.profile.json', f'{engine_name}.profile.metadata.json')

print(f"Summary for {plan.name}:\n")
plan.summary()

Summary for best_qat.onnx.engine.graph.json:

Model:
	Inputs: images: [1, 3, 640, 640]xFP32 NCHW
	Outputs: pred: [1, 80, 8400]xFP32 NCHW
	Average time: 7.369 ms
	Layers: 137
	Weights: 3.0 MB
	Activations: 100.5 MB
Device Properties:
	Selected Device: NVIDIA GeForce RTX 4070 SUPER
	Compute Capability: 8.9
	SMs: 56.0
	Device Global Memory: 12281 MiB
	Shared Memory per SM: 100 KiB
	Memory Bus Width: 192.0
	Application Compute Clock Rate: 2.475 GHz
	Application Memory Clock Rate: 10.501 GHz
Builder Configuration:
Performance Summary:
	Throughput: 1996.31
	Latency: [0.429932, 4.02026, 0.484867, 0.432129, 0.59903, 0.807861, 1.10229]
	Enqueue Time: [0.0185852, 4.30615, 0.119042, 0.041748, 0.487671, 0.552246, 0.829834]
	H2D Latency: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	GPU Compute Time: [0.429932, 4.02026, 0.484867, 0.432129, 0.59903, 0.807861, 1.10229]
	D2H Latency: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	Total Host Walltime: 3.00154
	Total GPU Compute Time: 2.90532


### 3.1. Layers Precision

In [81]:
charts = []
layer_precisions = trex.group_count(plan.df, 'precision')
charts.append((layer_precisions, 'Layer Count By Precision', 'count', 'precision'))

layers_time_pct_by_precision = trex.group_sum_attr(plan.df, grouping_attr='precision', reduced_attr='latency.pct_time')
display(layers_time_pct_by_precision)

charts.append((layers_time_pct_by_precision, '% Latency Budget By Precision', 'latency.pct_time', 'precision'))
trex.plotting.plotly_pie2("Precision Statistics", charts, colormap=trex.colors.precision_colormap);

,precision,latency.pct_time
0,FP16,6.623924
1,FP32,8.014247
2,INT8,85.361835


### 3.2. Graph Rendering

In [82]:
formatter = trex.graphing.layer_type_formatter if True else trex.graphing.precision_formatter
graph = trex.graphing.to_dot(plan, formatter)
svg_name = trex.graphing.render_dot(graph, engine_name, 'svg')
png_name = trex.graphing.render_dot(graph, engine_name, 'png')

Created file:///home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_silu/weights/tensorrt/best_qat.onnx.engine.svg
Created file:///home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_silu/weights/tensorrt/best_qat.onnx.engine.png


## 4. Inference

In [60]:
import torch
from memory_profiler import memory_usage
from ultralytics import YOLO
import time

# Determine if a GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
source = 'https://ultralytics.com/images/bus.jpg'

# Function to measure memory and run inference
def measure_memory_and_time(model_path, source):
    start_time = time.time()
    model = YOLO(model_path, task="detect")
    results = model.predict(source, device=device, save=True)
    end_time = time.time()
    inference_time = end_time - start_time
    print(f"Inference Time: {inference_time} seconds")
    return results, inference_time

### 4.1. Inference and Memory Profiling using PyTorch and ONNX

| Model                             | File            | Inference Time (seconds) |
|-----------------------------------|-----------------|--------------------------|
| Model Original + QAT in ONNX      | `best_qat.onnx` | 2.3577654361724854       |
| Model Original + QAT in PyTorch   | `best_qat.pt`   | 0.5823156833648682       |
| Model Original in PyTorch         | `best.pt`       | 0.5680086612701416       |


In [75]:
# Path to the model
model_path = '/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_relu/weights/best_qat.engine'
# model_path = '/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_silu/weights/best_qat.engine'
# model_path = "/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_relu/weights/best_qat.onnx"
# model_path = "/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_silu/weights/best.pt"
# model_path = "/home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_silu/weights/best_qat.pt"

# Measure initial memory usage
initial_memory = memory_usage(-1, interval=0.05, timeout=1)
print(f"Initial Memory Usage: {initial_memory} MB")

# Run inference and measure memory usage and inference time
results, inference_time = None, None
mem_usage = memory_usage(
    (measure_memory_and_time, (model_path, source)),
    interval=0.05,
    include_children=True,
    max_usage=False,
    retval=True,
)

# Extract results and memory usage
if isinstance(mem_usage, tuple):
    mem_usage, (results, inference_time) = mem_usage

# Measure final peak memory usage
final_peak_memory = max(mem_usage)

# Calculate average memory usage
average_memory = sum(mem_usage) / len(mem_usage)

print(f"Peak Memory Usage: {final_peak_memory} MB")
print(f"Average Memory Usage: {average_memory} MB")

Initial Memory Usage: [1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375, 1886.80859375] MB
Loading /home/jeffrymahbuubi/Syringe-Detection/compression/ultralytics-qat-jeffry/run/detect/QAT_b16_e100_relu/weights/best_qat.engine for TensorRT inference...
[07/05/2024-00:27:20] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[07/05/2024-00:27:20] [TRT] [I] Loaded engine size: 7 MiB
[07/05/2024-00:27:20] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +4, now: CPU 1, GPU 28 (MiB)
[07/05/2024-00:27:20] [TRT] [I] [MemUsageChange] TensorRT-man

### 4.2. Inference using `.engine` Format

##